In [14]:
# Import all required libraries

%matplotlib inline
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, classification_report, cohen_kappa_score, make_scorer
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
import random
from sklearn.datasets import load_digits
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold
import warnings
warnings.filterwarnings("ignore")
import scikitplot as skplt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB

In [15]:
# Import data set
col = ['Buying','Maint','Doors','Persons','Lug_Boot','Safety','Evaluation']
car = pd.read_csv("/Users/riyakhurana09/Desktop/Riya Khurana/Classes/Spring 2022/Predictive 6420/HW2/Q5/car.data", names = col)
car.head()

,Buying,Maint,Doors,Persons,Lug_Boot,Safety,Evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [16]:
# Defining Independent and Target variable
X = car.iloc[:,0:6]
y = np.where(car['Evaluation'] == 'unacc', 0, np.where(car['Evaluation'] == 'acc', 1, np.where(car['Evaluation'] == 'good', 2,3)))

### One-hot Encoding

In [17]:
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
X_one_hot = encoder.fit_transform(X)
X_one_hot

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 0., 0.]])

### Train-Test Split

In [18]:
# Split of 0.33 as test data
X_train, X_test, y_train, y_test = train_test_split(X_one_hot, y, test_size=0.33)

### Establishing Hyperparameters

In [19]:
# Decision Tree
dt_criterion = ['gini','entropy']
dt_splitter = ['best', 'random']
dt_max_depth = range(1,10)
dt_min_samples_leaf = range(1,10)

# KNN
knn_range = list(range(1,31))
knn_weights = ["uniform", "distance"]

# Logistic Regression
lr_solver = ['liblinear','saga', 'newton-cg','lbfgs']
lr_penalty = ['l1', 'l2']
lr_c_values = [ 10, 1, 0.1, 0.01,0.001]

# SVM
svm_kernel = ['rbf','linear']
svm_c_values = [1, 10, 100, 1000]
svm_gamma = [1e-3, 1e-4]

# Naive Bayes
nb_alpha = np.arange(-1,1,0.3)
nb_fit_prior = [False, True]

weight = ['balanced']

# Parameter Grid
dt_grid = dict(criterion = dt_criterion , splitter = dt_splitter, max_depth = dt_max_depth, \
               min_samples_leaf = dt_min_samples_leaf, class_weight = weight)
knn_grid = dict(n_neighbors = knn_range, weights = knn_weights)
lr_grid = dict(penalty = lr_penalty, solver = lr_solver, C = lr_c_values, class_weight = weight)
svm_grid = dict(kernel = svm_kernel, gamma = svm_gamma, C = svm_c_values, class_weight = weight)
nb_grid = dict(alpha = nb_alpha, fit_prior = nb_fit_prior)


In [20]:
# Cohen's Kappa Score
score = make_scorer(cohen_kappa_score)

# Classifier
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier(metric='hamming')
lr = LogisticRegression(max_iter = 10000)
svm = SVC()
nb = MultinomialNB()

### Grid Search

In [21]:
np.random.seed(11)

i = 1
inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

dt_clf = GridSearchCV(estimator = dt, scoring = score, param_grid = dt_grid, cv = inner_cv)
knn_clf = GridSearchCV(estimator = knn, scoring = score, param_grid = knn_grid, cv = inner_cv)
lr_clf = GridSearchCV(estimator = lr, scoring = score, param_grid = lr_grid, cv = inner_cv)
svm_clf = GridSearchCV(estimator = svm, scoring = score, param_grid = svm_grid, cv = inner_cv)
nb_clf = GridSearchCV(estimator = nb, scoring = score, param_grid = nb_grid, cv = inner_cv)

dt_score = cross_val_score(dt_clf, X = X_train, y = y_train, cv=outer_cv)
dt_score_mean = dt_score.mean()
knn_score = cross_val_score(knn_clf, X = X_train, y = y_train, cv=outer_cv)
knn_score_mean = knn_score.mean()
lr_score = cross_val_score(lr_clf, X = X_train, y = y_train, cv=outer_cv)
lr_score_mean = lr_score.mean()
svm_score = cross_val_score(svm_clf, X = X_train, y = y_train, cv=outer_cv)
svm_score_mean = svm_score.mean()
nb_score = cross_val_score(nb_clf, X = X_train, y = y_train, cv=outer_cv)
nb_score_mean = nb_score.mean()

### Model Results

In [25]:
print('Kappa Score \nDecision Tree:', round(dt_score_mean,3), '\nKNN:', round(knn_score_mean,3), '\nLogistic Regression:', \
      round(lr_score_mean,3), '\nSVM:', round(svm_score_mean,3), '\nNaive Bayes:', round(nb_score_mean,3))

Kappa Score 
Decision Tree: 0.87 
KNN: 0.757 
Logistic Regression: 0.84 
SVM: 0.838 
Naive Bayes: 0.664


### Best performing model - Decision Tree

In [23]:
# Decision Tree is our best performing model
dt_clf = GridSearchCV(estimator = dt, scoring = score, param_grid = dt_grid, cv = inner_cv)
dt_clf.fit(X_train, y_train)
print('Best Estimator (Hyperparameters):\n', dt_clf.best_estimator_)

Best Estimator (Hyperparameters):
 DecisionTreeClassifier(class_weight='balanced', max_depth=9, splitter='random')


### Classification Report

In [27]:
y_predict_test = dt_clf.predict(X_test)
target_names = ['Class 0','Class 1','Class 2','Class 3']
print('Confusion Matrix: \n',confusion_matrix(y_test,y_predict_test))
print('\nOverall Accuracy:\n',round(accuracy_score(y_test,y_predict_test),3))
print('\nClass 0: Unacceptable')
print('Class 1: Acceptable')
print('Class 2: Good')
print('Class 3: Vgood\n')
print(classification_report(y_test, y_predict_test, target_names=target_names))

Confusion Matrix: 
 [[395   9   0   0]
 [  1 113   1   4]
 [  2   0  26   0]
 [  0   0   0  20]]

Overall Accuracy:
 0.97

Class 0: Unacceptable
Class 1: Acceptable
Class 2: Good
Class 3: Vgood

              precision    recall  f1-score   support

     Class 0       0.99      0.98      0.99       404
     Class 1       0.93      0.95      0.94       119
     Class 2       0.96      0.93      0.95        28
     Class 3       0.83      1.00      0.91        20

    accuracy                           0.97       571
   macro avg       0.93      0.96      0.94       571
weighted avg       0.97      0.97      0.97       571



#### The Kappa Score of the best performing model - Decision Tree is 0.87 and the overall accuracy is 97.0%. Each class has been predicted differently which gives us a different score for each of them. The highest recall is achieved by vgood class which could be attributable to the number of observations in the class which is very low (very less support).